## 01. Gibbs-SeaWater (GSW) functions for Argo
---
**Author**: Hillary A. Scannell (scanh@uw.edu)

**Last Updated**: 25 November 2020

This notebook uses the Gibbs-SeaWater (GSW) Oceanographic Toolbox containing the TEOS-10 subroutines for evaluating the thermodynamic properties of seawater. Specifically we calculate: 

**A.** Absolute Salinity ($S_A$) from Argo practical salinity 

**B.** Conservative Temperature ($\theta$) from Argo in-situ temperature 

**C.** Potential Density Anomaly ($\sigma_t$) with reference pressure of 0 dbar using the calculated $S_A$ and $\theta$

Lastly, we compute anomalies in $S_A$, $\theta$, and $\sigma_t$ by removing the long-term (January 2004 – June 2020) monthly mean at each space and pressure point.


In [3]:
import xarray as xr
import numpy as np
import gsw
import matplotlib.pyplot as plt

In [4]:
print(f"xarray: {xr.__version__}")
print(f"numpy: {np.__version__}")

xarray: 0.16.1
numpy: 1.19.4


#### Import smoothed temperature and salinity fields from Argo.

In [9]:
# Gridded temperature and salinity fields are from the Reommich and 
# Gilson Argo Climatology. Using least square regression we remove the  
# mean, annual, and semiannual harmonics from January 2004 through June 2020. 
# we smooth the anomalies and the regression coefficients with a 5-month 
# Hanning filter and then a 6° latitude x 6° longitude LOESS filter. 
 
filepath = "/glade/scratch/scanh/grl_2020_data/te_se_smooth_RG09_Johnson_170720.zarr/"
ds = xr.open_zarr(filepath)

dates = np.arange('2004-01', '2020-07', dtype='datetime64[M]')
ds = ds.assign(dyr = dates)

# %time ds.load()

In [10]:
ds

<xarray.Dataset>
Dimensions:         (coeff: 6, dyr: 198, lat: 145, lon: 360, pressure: 58)
Coordinates:
  * coeff           (coeff) int64 1 2 3 4 5 6
  * dyr             (dyr) datetime64[ns] 2004-01-01 2004-02-01 ... 2020-06-01
  * lat             (lat) float64 -64.5 -63.5 -62.5 -61.5 ... 77.5 78.5 79.5
  * lon             (lon) float64 20.5 21.5 22.5 23.5 ... 377.5 378.5 379.5
  * pressure        (pressure) float64 2.5 10.0 20.0 ... 1.9e+03 1.975e+03
Data variables:
    argo_sa_mean    (pressure, lat, lon) float64 ...
    argo_te_mean    (pressure, lat, lon) float64 ...
    mask            (pressure, lat, lon) float64 ...
    sa_anom_smooth  (dyr, pressure, lat, lon) float64 ...
    sa_modelf       (coeff, pressure, lat, lon) float64 ...
    sa_tot          (dyr, pressure, lat, lon) float64 ...
    te_anom_smooth  (dyr, pressure, lat, lon) float64 ...
    te_modelf       (coeff, pressure, lat, lon) float64 ...
    te_tot          (dyr, pressure, lat, lon) float64 ...

---
**A.** Absolute Salinity ($S_A$) from Practical Salinity.

    gsw.SA_from_SP(SP, p, lon, lat)
    
Calculates Absolute Salinity from Practical Salinity. Since SP is non-negative by definition, this function changes any negative input values of SP to be zero.

**Parameters**:
- *SP* array-like Practical Salinity (PSS-78), unitless
- $p$ array-like Sea pressure (absolute pressure minus 10.1325 dbar), dbar
- $lon$ array-like Longitude, -360 to 360 degrees
- $lat$ array-like Latitude, -90 to 90 degrees

**Returns**:
- Absolute Salinity ($S_A$) array-like, g/kg Absolute Salinity

In [ ]:
%%time 
SA_tot = np.empty((ds.sa_tot.shape[0], ds.sa_tot.shape[1], ds.sa_tot.shape[2], ds.sa_tot.shape[3])) 
SA_tot[:] = np.nan

# Compute Absolute Salinity
for la in np.arange(0, ds.lat.shape[0]): 
    for lo in np.arange(0, ds.lon.shape[0]): 
        SA_tot[:,:,la,lo] = gsw.SA_from_SP(ds.sa_tot[:,:,la,lo].values, ds.pressure[:].values, 360-ds.lon[lo].values, ds.lat[la].values)


---
**B.** Conservative Temperature ($\Theta$) from in-situ temperature.
    
    gsw.CT_from_t(SA, t, p)

Calculates Conservative Temperature of seawater from in-situ temperature.

**Parameters**: 
- $S_A$ array-like Absolute Salinity, g/kg
- $t$ array-like In-situ temperature (ITS-90), degrees C
- $p$ array-like Sea pressure (absolute pressure minus 10.1325 dbar), dbar

**Returns**:
- Conservative Temperature ($\Theta$) array-like, deg C Conservative Temperature (ITS-90)



In [4]:
CT_tot = np.empty((ds.te_tot.shape[0], ds.te_tot.shape[1], ds.te_tot.shape[2], ds.te_tot.shape[3])) 
CT_tot[:] = np.nan

for p in np.arange(0, ds.pressure.shape[0]): 
    CT_tot[:,p,:,:] = gsw.CT_from_t(SA_tot[:,p,:,:], ds.te_tot[:,p,:,:].values, ds.pressure[p].values)


---
**C.** Potential Density Anomaly ($\sigma_t$) from absolute salinity and conservative temperature

    gsw.density.sigma0(SA, CT)

Calculates potential density anomaly with reference pressure of 0 dbar, this being this particular potential density minus 1000 kg/m^3. This function has inputs of Absolute Salinity and Conservative Temperature. This function uses the computationally-efficient expression for specific volume in terms of SA, CT and p (Roquet et al., 2015).

**Parameters**:
- $S_A$ array-like Absolute Salinity, g/kg
- $CT$ array-like Conservative Temperature (ITS-90), degrees C

**Returns**:
- Potential Density Anomaly ($\sigma_t$) array-like, kg/m$^{3}$ potential density anomaly with respect to a reference pressure of 0 dbar, that is, this potential density - 1000 kg/m$^{3}$.


In [5]:
sigmaT_tot = np.empty((ds.te_tot.shape[0], ds.te_tot.shape[1], ds.te_tot.shape[2], ds.te_tot.shape[3])) 
sigmaT_tot[:] = np.nan

for p in np.arange(0, ds.pressure.shape[0]): 
    sigmaT_tot[:,p,:,:] = gsw.density.sigma0(SA_tot[:,p,:,:], CT_tot[:,p,:,:])
    

---
**Compute anomalies in $S_A$, $\theta$, and $\sigma_t$** by removing the long-term (January 2004 – June 2020) monthly mean at each space and pressure point.

In [ ]:
# Initialize numpy matrices
SA_anom = np.empty((ds.sa_tot.shape[0], ds.sa_tot.shape[1], ds.sa_tot.shape[2], ds.sa_tot.shape[3])) 
SA_anom[:] = np.nan

CT_anom = np.empty((ds.te_tot.shape[0], ds.te_tot.shape[1], ds.te_tot.shape[2], ds.te_tot.shape[3])) 
CT_anom[:] = np.nan

sigmaT_anom = np.empty((ds.te_tot.shape[0], ds.te_tot.shape[1], ds.te_tot.shape[2], ds.te_tot.shape[3])) 
sigmaT_anom[:] = np.nan


In [7]:
for i in np.arange(1,13):
    I = np.where(ds.dyr.dt.month==i)[0]
    
    CT_mn = CT_tot[I,:,:,:].mean(axis=0)
    CT_anom[I,:,:,:] = CT_tot[I,:,:,:] - CT_mn
    
    SA_mn = SA_tot[I,:,:,:].mean(axis=0)
    SA_anom[I,:,:,:] = SA_tot[I,:,:,:] - SA_mn
    
    sigt_mn = sigmaT_tot[I,:,:,:].mean(axis=0)
    sigmaT_anom[I,:,:,:] = sigmaT_tot[I,:,:,:] - sigt_mn

---
**Save** the total and anomaly fields of $S_A$, $\theta$, and $\sigma_t$ to zarr

In [21]:
RG_filt_GSW = xr.Dataset({'SA_tot': (('dyr', 'pressure', 'lat', 'lon'), SA_tot),
                          'CT_tot': (('dyr', 'pressure', 'lat', 'lon'), CT_tot),
                          'sigmaT_tot': (('dyr','pressure', 'lat', 'lon'), sigmaT_tot),
                          'SA_anom': (('dyr', 'pressure', 'lat', 'lon'), SA_anom),
                          'CT_anom': (('dyr', 'pressure', 'lat', 'lon'), CT_anom),
                          'sigmaT_anom': (('dyr','pressure', 'lat', 'lon'), sigmaT_anom),
                          'mask': (('pressure', 'lat', 'lon'), ds.mask)},
                         coords={'dyr': ds.dyr, 
                                 'pressure': ds.pressure,
                                 'lat': ds.lat,
                                 'lon': ds.lon})

RG_filt_GSW.to_zarr("/glade/scratch/scanh/grl_2020_data/CT_SA_sigmaT_smooth_RG09_Johnson_170720.zarr")
